## Ancestry Seach 

<b>Goal: </b>Multiple different spellings of a name can be referring to the same identity. We will use a phonetics library and Ancestry to fix this. An example: ```David Schaffer``` and ```David Schafer``` from `MA`. 

<b>Steps: </b>
1. Login to Emory's Ancestry subscription 
2. Iterate through ```agg_debt```, through each debt entry. 
3. Use a combination of phonetics fuzzy string matching and normal fuzzy string matching to determine if two names from a state are similar.  
4. Search each name in Ancestry: Edit URL (state and person's name). 
5. Check if there are any results for both person's name:
    - Yes: Check if one spelling of the name appears for both individuals (that's most likely the correct spelling of that name) 
    - No: Leave entries as two separate entries. 
6. Record name change in ```fixes``` list (save ```fixes``` as ```out.csv``` too). 
7. Run ```agg_debt``` through ```fixes```, making changes as necessary. 
8. Save ```agg_debt``` as a new .csv file.

<b style="color: red;">Note: Runtime is long. This is due to the fact there are over 200,000 debt entries and accessing Ancestry takes time too. </b>

In [1]:
# import necessary fuzzy string libraries 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.expected_conditions import element_to_be_clickable, presence_of_element_located
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from phonetics import metaphone
from rapidfuzz import fuzz
from joblib import Parallel, delayed, cpu_count
from itertools import zip_longest
import time 
import getpass
import pickle
import pandas as pd
import numpy as np
import ast

In [96]:
agg_debt = pd.read_csv('data/agg_debt_grouped_updated.csv')

C:\Users\david\AppData\Local\Temp\ipykernel_49640\2714780960.py:1: DtypeWarning: Columns (2,10,11,13,15,16,20,21,22,24,31) have mixed types. Specify dtype option on import or set low_memory=False.
  agg_debt = pd.read_csv('data/agg_debt_grouped_updated.csv')


In [97]:
# Find number of unique individuals in `agg_debt`
agg_debt.groupby(['to whom due | first name', 'to whom due | last name']).ngroups 

65734

In [89]:
agg_debt['to whom due | first name'] = agg_debt['to whom due | first name'].astype(str)
agg_debt['to whom due | last name'] = agg_debt['to whom due | last name'].astype(str)

In [4]:
name_changes = pd.read_csv('data/name_changes_david.csv')

In [5]:
agg_debt.head()

,Unnamed: 0,index,letter,date of the certificate | month,date of the certificate | day,date of the certificate | year,to whom due | first name,to whom due | last name,to whom due | title,time when the debt became due | month,...,amount | 10th,exchange,amount in specie | dollars,amount in specie | cents,amount | 8th,delivered | month,delivered | day,delivered | year,total dollars | notes,total dollars | notes.1
0,0,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.0,C,8.0,27.0,1783.0,Elizabeth,Lowell,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2.0,G,8.0,27.0,1783.0,Joshua,Brackett,Esq,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3.0,C,9.0,2.0,1783.0,Phillips,White,Esq,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4.0,B,9.0,2.0,1783.0,William,White,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Check to make sure pierce certificate first name and last name columns are properly swapped
agg_debt.loc[agg_debt['org_file'] == 'Pierce_Certs_cleaned_2019.xlsx'].head()

,Unnamed: 0,index,letter,date of the certificate | month,date of the certificate | day,date of the certificate | year,to whom due | first name,to whom due | last name,to whom due | title,time when the debt became due | month,...,amount | 10th,exchange,amount in specie | dollars,amount in specie | cents,amount | 8th,delivered | month,delivered | day,delivered | year,total dollars | notes,total dollars | notes.1
14015,14015,15296.0,NaN,NaN,NaN,NaN,Mason,Abbe,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14016,14016,15298.0,NaN,NaN,NaN,NaN,John,Abbee,Jun,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14017,14017,15299.0,NaN,NaN,NaN,NaN,Richard,Abbet,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14018,14018,15300.0,NaN,NaN,NaN,NaN,Reuben,Abbett,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14019,14019,15301.0,NaN,NaN,NaN,NaN,Richard,Abbett,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
name_changes.tail()

,Unnamed: 0,title_org,title_new,first_name_org,last_name_org,first_name_new,last_name_new,cleaning case,file_loc,org_index
1627,1627,NaN,NaN,Jn,Gallaher Esq,John,Gallaher Esq,5,loan_office_certificates_9_states.xlsx,62583
1628,1628,NaN,NaN,Rich,Thompson,Richard,Thompson,5,loan_office_certificates_9_states.xlsx,69947
1629,1629,NaN,NaN,Rich,Thompson,Richard,Thompson,5,loan_office_certificates_9_states.xlsx,69948
1630,1630,NaN,NaN,Rich,Thompson,Richard,Thompson,5,loan_office_certificates_9_states.xlsx,69949
1631,1631,NaN,NaN,Rich,Thompson,Richard,Thompson,5,loan_office_certificates_9_states.xlsx,69950


In [8]:
# Options
options = Options()
options.add_argument('--headless')
options.add_argument("--window-size=1000,1000")
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--no-sandbox')   

In [9]:
agg_debt.state.unique()

array([nan, 'nh', 'nj', 'ny', 'ma', 'de', 'ct', 'va', 'pa', 'md', 'nc',
       'cs', 'ga', 'ri', 'f'], dtype=object)

In [10]:
# Voter records and censuses available for every state 
records = {
    'nh':['https://www.ancestrylibrary.com/search/collections/5058/'],
    'nj':['https://www.ancestrylibrary.com/search/collections/2234/', 
          'https://www.ancestrylibrary.com/search/collections/3562/'],
    'ny':['https://www.ancestrylibrary.com/search/collections/5058/'],
    'ma':['https://www.ancestrylibrary.com/search/collections/5058/'], 
    'ct':['https://www.ancestrylibrary.com/search/collections/5058/'], 
    'va':['https://www.ancestrylibrary.com/search/collections/2234/', 
         'https://www.ancestrylibrary.com/search/collections/3578/'], 
    'pa':['https://www.ancestrylibrary.com/search/collections/2702/',
         'https://www.ancestrylibrary.com/search/collections/2234/',
         'https://www.ancestrylibrary.com/search/collections/3570/'],
    'md':['https://www.ancestrylibrary.com/search/collections/3552/'],
    'nc':['https://www.ancestrylibrary.com/search/collections/3005/', 
         'https://www.ancestrylibrary.com/search/collections/2234/'],
    'ga':['https://www.ancestrylibrary.com/search/collections/2234/'],
    'ri':['https://www.ancestrylibrary.com/search/collections/3571/']
}

# Ancestry has unique urls for each state
residence_urls = {
    'nh':'_new+hampshire-usa_32',
    'nj':'_new+jersey-usa_33', 
    'ny':'_new+york-usa_35',
    'ma':'_massachusetts-usa_24',
    'ct':'_connecticut-usa_9',
    'va':'_virginia-usa_49', 
    'pa':'_pennsylvania-usa_41',
    'md':'_maryland-usa_23',
    'nc':'_north+carolina-usa_36',
    'ga':'_georgia-usa_13',
    'ri':'_rhode+island-usa_42'
}

In [11]:
# Remove 'cs' (congress) and 'f' (foreign officers); these are not state, but specific regiments / types of officers
agg_debt_copy = agg_debt[(agg_debt['state'] != 'cs') & (agg_debt['state'] != 'f') & (agg_debt['state'] != 'de')]

ct
ga
ma
md
nc
nh
nj
ny
pa
ri
va


In [12]:
# Store all similar_names files as Pandas Dataframes
similar_names_dfs = {}
for state in agg_debt_sp.groups:
    similar_names_dfs[state] = pd.read_csv('data/similar names/similar_names_' + state + '.csv') 

print(len(similar_names_dfs))

11


In [13]:
netid_xpath = '/html/body/div[1]/div[2]/section/div[1]/div/form/fieldset/div[1]/input'
password_xpath = '/html/body/div[1]/div[2]/section/div[1]/div/form/fieldset/div[2]/input'
login_btn0_xpath = '/html/body/main/div/div/div/a'
login_btn1_xpath = '/html/body/div[1]/div[2]/section/div[1]/div/form/fieldset/div[3]/button'

# Ask for a password and username 
username = input('username: ')
password = getpass.getpass(prompt='password: ')

driver_objs = {}

# Create a new driver object for each state
for i in range(cpu_count()):
    webdriver_obj = webdriver.Chrome(service=Service(executable_path="chromedriver.exe"), options=options)
    driver_objs[i] = (webdriver_obj, WebDriverWait(webdriver_obj, 30))

'''
# create a new wait object for each state
for st in cpu_count:
    webdriver_obj = driver_objs[st][0]
    driver_objs[st].append(WebDriverWait(webdriver_obj, 30))
'''
    
# For each driver object, access emory's ancestry's subscription 
for i in range(cpu_count()):
    webdriver_obj = driver_objs[i][0]
    wait_obj = driver_objs[i][1]
    
    # Go to Emory's library 
    webdriver_obj.get('https://guides.libraries.emory.edu/ALE')
    wait_obj.until(element_to_be_clickable((By.XPATH, login_btn0_xpath))).click()
    
    # Input login information and click 'login'
    netid_input = wait_obj.until(element_to_be_clickable((By.XPATH, netid_xpath)))
    netid_input.click()
    netid_input.send_keys(username)
    pass_input = wait_obj.until(element_to_be_clickable((By.XPATH, password_xpath)))
    pass_input.click()
    pass_input.send_keys(password) 
    wait_obj.until(element_to_be_clickable((By.XPATH, login_btn1_xpath))).click()
    time.sleep(1)
    
    webdriver_obj.get('https://www.ancestrylibrary.com/search/collections/5058/')
    print(webdriver_obj.current_url)
    
    print(webdriver_obj)

username: dcho52
password: ········
https://www.ancestrylibrary.com/search/collections/5058/
<selenium.webdriver.chrome.webdriver.WebDriver (session="be8ee6eee56605128927fbda4c0f078e")>
https://www.ancestrylibrary.com/search/collections/5058/
<selenium.webdriver.chrome.webdriver.WebDriver (session="5b6f54551c2effc31dea3b2046092aba")>
https://www.ancestrylibrary.com/search/collections/5058/
<selenium.webdriver.chrome.webdriver.WebDriver (session="35fd95d190d8ad6f0a6df7cd3b2ba197")>
https://www.ancestrylibrary.com/search/collections/5058/
<selenium.webdriver.chrome.webdriver.WebDriver (session="23aec69e1c6288db0f32caf97632748a")>
https://www.ancestrylibrary.com/search/collections/5058/
<selenium.webdriver.chrome.webdriver.WebDriver (session="fadcd696f5433b35b785c3bc86bc81ca")>
https://www.ancestrylibrary.com/search/collections/5058/
<selenium.webdriver.chrome.webdriver.WebDriver (session="2ddf264882e3cb208a08b42d588b0bf8")>
https://www.ancestrylibrary.com/search/collections/5058/
<seleni

In [14]:
print(len(driver_objs))

7


## Accessing Ancestry

In [14]:
# When running this code for the first time, set all variables equal to '[]'

'''
ancestry_name_changes = []
rerun_rows = []
checked = []
fixes = []
'''

'\nancestry_name_changes = []\nrerun_rows = []\nchecked = []\nfixes = []\n'

In [15]:
%store -r ancestry_name_changes
%store -r rerun_rows
%store -r checked
%store -r fixes

In [16]:
def access_ancestry(row0, state, driver, wait):
    fn0 = str(row0['to whom due | first name'])
    ln0 = str(row0['to whom due | last name'])
    matches = ast.literal_eval(row0['matches'])
    name0 = fn0 + ' ' + ln0

    for match in matches:
        row1 = agg_debt.loc[[match[2]]] 
        fn1 = str(row1['to whom due | first name'].values[0])
        ln1 = str(row1['to whom due | last name'].values[0])
        name1 = fn1 + ' ' + ln1
        if ((name0, name1, state) not in checked and (name1, name0, state) not in checked): 
            search_ancestry(fn0, ln0, fn1, ln1, row0, row1, state, driver, wait)
            checked.append((name0, name1, state))
            %store checked

In [17]:
# Look up both names in Ancestry's database
def search_ancestry(fn0, ln0, fn1, ln1, row0, row1, state, driver, wait):
    name0 = fn0 + ' ' + ln0
    name1 = fn1 + ' ' + ln1
    
    # Loop through state urls 
    for url in records[state]:        
        try:
            # Search for person 0
            url0 = url + '?name=' + fn0 + '_' + ln0 + '&name_x=ps&residence=1780' + residence_urls[state] + '&residence_x=10-0-0_1-0'
            driver.get(url0) 
            # Results were found for person 0
            try:
                # Use class_name to find result text
                result0 = wait.until(presence_of_element_located((By.CLASS_NAME, 'srchHit'))).text
            # No results were found; Keep entries separate  
            except:
                result0 = ''
            
            # Search for person 1
            url1 = url + '?name=' + fn1 + '_' + ln1 + '&name_x=ps&residence=1780' + residence_urls[state] + '&residence_x=10-0-0_1-0'
            driver.get(url1) 
            # results were found for person1
            try: 
                # use class_name to find result text
                result1 = wait.until(presence_of_element_located((By.CLASS_NAME, 'srchHit'))).text
            # no results were found; keep entries separate
            except:
                result1 = ''    

            '''
            compare results:
            if both results are empty, do not add to fixes dict 
            if both results are different, do not add to fixes dict
            if both results are the same, add to fixes dict
                find correct name
                if name0 = result0 and result1 : {name1 : name0}
                if name1 = result1 and result0 : {name0 : name0} 
            '''
            
            print('---------------------------+')
            if result0 == result1 and result0 != '' and result1 != '':
                if name0 == result0 and name0 == result1: # name0 must be the correct version of the name 
                    # record change
                    title1 = row1['to whom due | title']
                    org_file1 = row1['org_file']
                    org_index1 = row1['org_index'] 
                    ancestry_name_changes.append([title1, title1, fn1, ln1, fn0, ln0, 6, org_file1, org_index1, state])
                    %store ancestry_name_changes
                    fixes.append({fn1: fn0, ln1: ln0, 'state': state})
                    %store fixes

                elif name1 == result0 and name1 == result1: # name1 must be the correct version of the name 
                    # record change
                    title0 = row0['to whom due | title']
                    org_file0 = row0['org_file']
                    org_index0 = row0['org_index'] 
                    ancestry_name_changes.append([title0, title0, fn0, ln0, fn1, ln1, 6, org_file0, org_index0, state])
                    %store ancestry_name_changes
                    fixes.append({fn0: fn1, ln0: ln1, 'state': state})
                    %store fixes

            print('Summary')
            print('name0=' + str(name0))
            print('name1=' + str(name1))
            print('url-0=' + str(url0))
            print('url-1=' + str(url1))
            print('result0=' + str(result0))
            print('result1=' + str(result1))
            print('state=' + str(state))

            '''
            print('fn0=' + str(fn0))
            print('ln0=' + str(ln0))
            print('fn1=' + str(fn1))
            print('ln1=' + str(ln1))
            '''
            print('index=' + str(row0['index']))
            print('rerun_rows length=' + str(len(rerun_rows)))
            print('name_changes length=' + str(len(ancestry_name_changes)))
            print('fixes length=' + str(len(fixes)))
            '''
            print('file_name0=' + str(org_file0))
            print('file_name1=' + str(org_file1))
            '''
            print('---------------------------+')
        
        # there was error trying to access ancestry's records
        except Exception as e:
            print('---------------------------+')
            print('Error')
            print(e)
            '''
            print('name0=' + str(name0))
            print('name1=' + str(name1))
            print('title1=' + str(type(title1)))
            print('title0=' + str(type(title0)))
            print('title1=' + str(title1))
            print('title0=' + str(title0))
            print('org_file1=' + str(org_file1))
            print('org_file0=' + str(org_file0))
            print('index=' + str(index))
            '''
            print('---------------------------+')
            if (row0, row1) not in rerun_rows:
                rerun_rows.append((row0, row1)) 
                %store rerun_rows

In [18]:
print(len(ancestry_name_changes))
print(len(rerun_rows))

2193


In [29]:
# Merge all the separate similar_names files into one larger DataFrame
similar_names = []
for state in agg_debt_sp.groups:
    similar_names.append(similar_names_dfs[state])
similar_names_df = pd.concat(similar_names)
print(len(similar_names_df))

[      Unnamed: 0.1  Unnamed: 0     index letter  \
0            14308       14308   15993.0    NaN   
1            14319       14319   16020.0    NaN   
2            14431       14431   16400.0    NaN   
3            14480       14480   16508.0    NaN   
4            14510       14510   16562.0    NaN   
...            ...         ...       ...    ...   
1487         73241       73241  147819.0    NaN   
1488         73242       73242  147820.0    NaN   
1489         73243       73243  147821.0    NaN   
1490         73248       73248  147832.0    NaN   
1491         73255       73255  147857.0    NaN   

      date of the certificate | month  date of the certificate | day  \
0                                 NaN                            NaN   
1                                 NaN                            NaN   
2                                 NaN                            NaN   
3                                 NaN                            NaN   
4                         

In [30]:
def ancestry_wrap(similar_names, driver_wait):    
    similar_names.apply(lambda row0: access_ancestry(row0, row0['state'], driver_wait[0], driver_wait[1]), axis=1) 

In [32]:
df_split = np.array_split(similar_names_df, cpu_count())  
print(len(df_split))

# Initialize a parallelization job 
ancestry_calls = [delayed(ancestry_wrap)(df_split[i], driver_objs[i]) for i in range(7)]
results = Parallel(n_jobs=-1, backend="threading")(ancestry_calls) 

16


In [20]:
# Due to the number of entries in Pennsylvania, I decided to parallelize on PA's data only. 
split_pa = np.array_split(similar_names_dfs['pa'], cpu_count())
print(len(split_pa))

7


In [23]:
# Initialize a parallelization job 
ancestry_calls = [delayed(ancestry_wrap)(split_pa[i], driver_objs[i]) for i in range(cpu_count())]
results = Parallel(n_jobs=-1, backend="threading")(ancestry_calls) 

---------------------------+
Summary
name0=George Haff
name1=George George
url-0=https://www.ancestrylibrary.com/search/collections/2702/?name=George_Haff&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/2702/?name=George_George&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=George Hoff
result1=George Powel 
state=pa
index=180237.0
rerun_rows length=24
name_changes length=2193
fixes length=2193
---------------------------+
---------------------------+
Summary
name0=William Searse
name1=William S
url-0=https://www.ancestrylibrary.com/search/collections/2702/?name=William_Searse&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/2702/?name=William_S&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=
result1=
state=pa
index=172806.0
rerun_rows length=24
name_changes length=2193
fixes len

---------------------------+
Summary
name0=John Kolber
name1=John Solber
url-0=https://www.ancestrylibrary.com/search/collections/3570/?name=John_Kolber&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/3570/?name=John_Solber&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=
result1=
state=pa
index=180245.0
rerun_rows length=24
name_changes length=2193
fixes length=2193
---------------------------+
Stored 'checked' (list)
---------------------------+
Summary
name0=Jacob Dyer
name1=Jacob Myer
url-0=https://www.ancestrylibrary.com/search/collections/2234/?name=Jacob_Dyer&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/2234/?name=Jacob_Myer&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=
result1=Jacob Myre
state=pa
index=178966.0
rerun_rows length=24
name_changes length=2193
fixes len

---------------------------+
Summary
name0=Richard Samers
name1=Richard Somers
url-0=https://www.ancestrylibrary.com/search/collections/2702/?name=Richard_Samers&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/2702/?name=Richard_Somers&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=
result1=
state=pa
index=180351.0
rerun_rows length=24
name_changes length=2194
fixes length=2194
---------------------------+
---------------------------+
Summary
name0=Pierre Despres
name1=Pierce Despres
url-0=https://www.ancestrylibrary.com/search/collections/3570/?name=Pierre_Despres&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/3570/?name=Pierce_Despres&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=
result1=
state=pa
index=179021.0
rerun_rows length=24
name_changes length=2194
fixes length=21

---------------------------+
Summary
name0=James Hurton
name1=James Huston
url-0=https://www.ancestrylibrary.com/search/collections/3570/?name=James_Hurton&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/3570/?name=James_Huston&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=
result1=
state=pa
index=180357.0
rerun_rows length=24
name_changes length=2195
fixes length=2195
---------------------------+
Stored 'checked' (list)
---------------------------+
Summary
name0=John Machey
name1=John Mackey
url-0=https://www.ancestrylibrary.com/search/collections/3570/?name=John_Machey&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/3570/?name=John_Mackey&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=
result1=John Mc Kay
state=pa
index=179189.0
rerun_rows length=24
name_changes length=2195


---------------------------+
Summary
name0=David Corney
name1=David Correy
url-0=https://www.ancestrylibrary.com/search/collections/2702/?name=David_Corney&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/2702/?name=David_Correy&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=
result1=David Correy
state=pa
index=180444.0
rerun_rows length=24
name_changes length=2195
fixes length=2195
---------------------------+
---------------------------+
Summary
name0=David Corney
name1=David Correy
url-0=https://www.ancestrylibrary.com/search/collections/2234/?name=David_Corney&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/2234/?name=David_Correy&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=
result1=David Curry
state=pa
index=180444.0
rerun_rows length=24
name_changes length=2195
fixes le

...

---------------------------+
Summary
name0=Ernest Clinger
name1=Ernest Klinger
url-0=https://www.ancestrylibrary.com/search/collections/2234/?name=Ernest_Clinger&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/2234/?name=Ernest_Klinger&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=
result1=
state=pa
index=173477.0
rerun_rows length=25
name_changes length=2211
fixes length=2211
---------------------------+
---------------------------+
Summary
name0=Christian Lowe
name1=Christian Lower
url-0=https://www.ancestrylibrary.com/search/collections/2702/?name=Christian_Lowe&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/2702/?name=Christian_Lower&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=
result1=Christn Lower
state=pa
index=180120.0
rerun_rows length=25
name_changes length=2211


---------------------------+
Summary
name0=Christian Lewer
name1=Christian Loeer
url-0=https://www.ancestrylibrary.com/search/collections/2702/?name=Christian_Lewer&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/2702/?name=Christian_Loeer&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=Christian Lawer
result1=Christian Lawer
state=pa
index=180136.0
rerun_rows length=25
name_changes length=2211
fixes length=2211
---------------------------+
---------------------------+
Summary
name0=Rudolph Schreiner
name1=Rudolph Schrine
url-0=https://www.ancestrylibrary.com/search/collections/3570/?name=Rudolph_Schreiner&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/3570/?name=Rudolph_Schrine&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=
result1=
state=pa
index=173500.0
rerun_rows length=2

---------------------------+
Summary
name0=Robert Steven
name1=Robert Stevens
url-0=https://www.ancestrylibrary.com/search/collections/2702/?name=Robert_Steven&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/2702/?name=Robert_Stevens&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=Robert Stephens
result1=Robert Stephens
state=pa
index=173537.0
rerun_rows length=25
name_changes length=2211
fixes length=2211
---------------------------+
---------------------------+
Summary
name0=Robert Steven
name1=Robert Stevens
url-0=https://www.ancestrylibrary.com/search/collections/2234/?name=Robert_Steven&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/2234/?name=Robert_Stevens&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=Robert Stephen
result1=Robert Stephen
state=pa
index=173537.0
rerun_r

---------------------------+
Summary
name0=Henry Mower
name1=Henry Hower
url-0=https://www.ancestrylibrary.com/search/collections/2234/?name=Henry_Mower&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/2234/?name=Henry_Hower&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=
result1=
state=pa
index=173552.0
rerun_rows length=25
name_changes length=2211
fixes length=2211
---------------------------+
---------------------------+
Summary
name0=Henry Mower
name1=Henry Hower
url-0=https://www.ancestrylibrary.com/search/collections/3570/?name=Henry_Mower&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/3570/?name=Henry_Hower&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=Henry Meyer
result1=Henry Howard
state=pa
index=173552.0
rerun_rows length=25
name_changes length=2211
fixes length=221

---------------------------+
Summary
name0=Rem Cornel
name1=Rem Cornell
url-0=https://www.ancestrylibrary.com/search/collections/3570/?name=Rem_Cornel&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/3570/?name=Rem_Cornell&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=
result1=
state=pa
index=173681.0
rerun_rows length=25
name_changes length=2211
fixes length=2211
---------------------------+
Stored 'checked' (list)
---------------------------+
Summary
name0=George Lownes
name1=George George
url-0=https://www.ancestrylibrary.com/search/collections/2702/?name=George_Lownes&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
url-1=https://www.ancestrylibrary.com/search/collections/2702/?name=George_George&name_x=ps&residence=1780_pennsylvania-usa_41&residence_x=10-0-0_1-0
result0=George Lownes
result1=George Powel 
state=pa
index=173693.0
rerun_rows length=25
name_cha

In [27]:
# Store the `ancestry_name_changes` list as a Pandas DataFrame
ancestry_df = pd.DataFrame(ancestry_name_changes)
# Store DataFrame as a .csv file 
ancestry_df.to_csv('data/fixes.csv')

## Cleaning Name Changes

In [33]:
# Remove duplicates
print(len(ancestry_name_changes))
res = []
checked = []
for row in ancestry_name_changes:
    if (row[2], row[3], row[4], row[5], row[9]) not in checked:
        res.append(row)
        checked.append((row[2], row[3], row[4], row[5], row[9]))

print(len(res))

2211
1946


In [50]:
# Remove metadata 
for row in res:
    if (type(row[0]) == pd.core.series.Series and type(row[1]) == pd.core.series.Series and type(row[7]) == pd.core.series.Series and
       type(row[8]) == pd.core.series.Series):
        row[0] = row[0].values[0]
        row[1] = row[1].values[0]
        row[7] = row[7].values[0]
        row[8] = row[8].values[0]

In [55]:
print(res[2])

['Rev', 'Rev', 'Thomas', 'Allyn', 'Thomas', 'Allen', 6, 'loan_office_certificates_9_states.xlsx', 18826.0, 'ct']


In [56]:
# Standardize datatypes for each row in each column
for row in res:
    row[2] = str(row[2])
    row[3] = str(row[3])
    row[4] = str(row[4])
    row[5] = str(row[5])
    row[6] = int(row[6])
    row[7] = str(row[7])
    row[8] = int(row[8])
    row[9] = str(row[9])

In [57]:
print(res[2])

['Rev', 'Rev', 'Thomas', 'Allyn', 'Thomas', 'Allen', 6, 'loan_office_certificates_9_states.xlsx', 18826, 'ct']


In [59]:
ancestry_name_changes = res
# Store the `fixes` dictionary as a Pandas DataFrame
ancestry_df = pd.DataFrame(ancestry_name_changes)
ancestry_df.columns = ['Old Title', 'New Title', 'Old First Name', 'Old Last Name', 'New First Name', 'New Last Name', 
                       'Obj. Num.', 'Original File', 'Original Index', 'State']
# Store DataFrame as a .csv file 
ancestry_df.to_csv('data/ancestry_name_changes_clean.csv')

## Add Ancestry Name Changes to the Name Changes Table

In [64]:
name_changes.drop(columns=['Unnamed: 0'], inplace=True)

In [65]:
name_changes.head()

,title_org,title_new,first_name_org,last_name_org,first_name_new,last_name_new,cleaning case,file_loc,org_index
0,NaN,NaN,Henry Wisner & Co,NaN,Henry,Wisner,2,liquidated_debt_certificates_NY.xlsx,491
1,NaN,NaN,Henry Wisner & Co,NaN,Henry,Wisner,2,liquidated_debt_certificates_NY.xlsx,492
2,NaN,NaN,Henry Wisner & Co,NaN,Henry,Wisner,2,liquidated_debt_certificates_NY.xlsx,493
3,NaN,NaN,Henry Wisner & Co,NaN,Henry,Wisner,2,liquidated_debt_certificates_NY.xlsx,494
4,NaN,NaN,Henry Wisner & Co,NaN,Henry,Wisner,2,liquidated_debt_certificates_NY.xlsx,495


In [66]:
# Add Ancestry name changes to name_changes file 
def add_changes(title_org, title_new, fn_org, ln_org, fn_new, ln_new, case, file, index):
    new_row = [title_org, title_new, fn_org, ln_org, fn_new, ln_new, case, file, index]
    name_changes.loc[len(name_changes.index)] = new_row

In [67]:
# Add Ancestry name changes to name_changes file 
for row in res:
    add_changes(row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8]) 

In [68]:
name_changes.tail()

,title_org,title_new,first_name_org,last_name_org,first_name_new,last_name_new,cleaning case,file_loc,org_index
3573,NaN,NaN,John,Krawson,John,Krewson,6,loan_office_certificates_9_states.xlsx,57961
3574,NaN,NaN,John,Luine,John,Line,6,loan_office_certificates_9_states.xlsx,72169
3575,Aug,Aug,Alexander,Browne,Alexander,Brown,6,loan_office_certificates_9_states.xlsx,57997
3576,NaN,NaN,Henry,Shoffer,Henry,Sheffer,6,loan_office_certificates_9_states.xlsx,51816
3577,Mr,Mr,Joseph,Walkar,Joseph,Walker,6,loan_office_certificates_9_states.xlsx,58409


In [69]:
# Save name_changes 
name_changes.to_csv('data/name_changes_david.csv')

## Create a Final Debt Table

In [90]:
def fix_names(row):
    search_res = ancestry_df.loc[(ancestry_df['Old First Name'] == row['to whom due | first name']) & (ancestry_df['Old Last Name'] == row['to whom due | last name']) & (ancestry_df['State'] == row['state'])]
    if len(search_res != 0): 
        print(row['to whom due | first name'])
        print(row['to whom due | last name'])
        row['to whom due | first name'] = search_res['New First Name']
        row['to whom due | last name'] = search_res['New Last Name']
        print('+------------------+')
        print(row['to whom due | first name'])
        print(row['to whom due | last name'])
    return row

In [91]:
agg_debt.head()

,Unnamed: 0,index,letter,date of the certificate | month,date of the certificate | day,date of the certificate | year,to whom due | first name,to whom due | last name,to whom due | title,time when the debt became due | month,...,amount | 10th,exchange,amount in specie | dollars,amount in specie | cents,amount | 8th,delivered | month,delivered | day,delivered | year,total dollars | notes,total dollars | notes.1
0,0,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.0,C,8.0,27.0,1783.0,Elizabeth,Lowell,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2.0,G,8.0,27.0,1783.0,Joshua,Brackett,Esq,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3.0,C,9.0,2.0,1783.0,Phillips,White,Esq,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4.0,B,9.0,2.0,1783.0,William,White,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
print(type(agg_debt))

<class 'pandas.core.frame.DataFrame'>


In [93]:
agg_debt = agg_debt.apply(lambda row: fix_names(row), axis=1)

Winthrop
Wiggins
+------------------+
20    Winthrop
Name: New First Name, dtype: object
20    Wiggin
Name: New Last Name, dtype: object
Stacey
Potts
+------------------+
4    Stacy
Name: New First Name, dtype: object
4    Potts
Name: New Last Name, dtype: object
Samuel
Coole
+------------------+
25    Samuel
Name: New First Name, dtype: object
25    Cole
Name: New Last Name, dtype: object
Samuel
Tuker
+------------------+
64    Samuel
Name: New First Name, dtype: object
64    Tucker
Name: New Last Name, dtype: object
David
Johnes
+------------------+
67    David
Name: New First Name, dtype: object
67    Jones
Name: New Last Name, dtype: object
Jonathan
Hatchinson
+------------------+
103    Jonathan
Name: New First Name, dtype: object
103    Hutchinson
Name: New Last Name, dtype: object
William
Seman
+------------------+
106    William
Name: New First Name, dtype: object
106    Seaman
Name: New Last Name, dtype: object
Isaac
Peirson
+------------------+
118    Isaac
Name: New First Na

Jacob
VanDike
+------------------+
627    Jacob
Name: New First Name, dtype: object
627    Vandike
Name: New Last Name, dtype: object
Rynier
Veghte
+------------------+
624    Reynier
Name: New First Name, dtype: object
624    Veghte
Name: New Last Name, dtype: object
Peter A
Dumont
+------------------+
643    Peter
Name: New First Name, dtype: object
643    Dumont
Name: New Last Name, dtype: object
William
VanDoren
+------------------+
646    William
Name: New First Name, dtype: object
646    Vandorn
Name: New Last Name, dtype: object
Moses
Scot
+------------------+
650    Moses
Name: New First Name, dtype: object
650    Scott
Name: New Last Name, dtype: object
William
Bladwin
+------------------+
457    William
Name: New First Name, dtype: object
457    Baldwin
Name: New Last Name, dtype: object
David
Hayes
+------------------+
659    David
Name: New First Name, dtype: object
659    Hays
Name: New Last Name, dtype: object
Rolph
Morsh
+------------------+
85    Rolph
Name: New First N

John
Fishear
+------------------+
70    John
Name: New First Name, dtype: object
70    Fisher
Name: New Last Name, dtype: object
Theunis
Cooper
+------------------+
36    Teunis
Name: New First Name, dtype: object
36    Cooper
Name: New Last Name, dtype: object
Josiah
Scely
+------------------+
86    Josiah
Name: New First Name, dtype: object
86    Seely
Name: New Last Name, dtype: object
Josiah
Scely
+------------------+
86    Josiah
Name: New First Name, dtype: object
86    Seely
Name: New Last Name, dtype: object
John L
Moffat
+------------------+
93    John
Name: New First Name, dtype: object
93    Moffat
Name: New Last Name, dtype: object
George
Thomson
+------------------+
98    George
Name: New First Name, dtype: object
98    Thompson
Name: New Last Name, dtype: object
Samson
Dyckman
+------------------+
102    Sampson
Name: New First Name, dtype: object
102    Dyckman
Name: New Last Name, dtype: object
Archabald
Stewart
+------------------+
105    Archibald
Name: New First Name

Peter
Woormwood
+------------------+
422    Peter
Name: New First Name, dtype: object
422    Wormwood
Name: New Last Name, dtype: object
John
Sanderse
+------------------+
357    John
Name: New First Name, dtype: object
357    Sanders
Name: New Last Name, dtype: object
John
Sanderse
+------------------+
357    John
Name: New First Name, dtype: object
357    Sanders
Name: New Last Name, dtype: object
Nicholas
Veider
+------------------+
370    Nicholas
Name: New First Name, dtype: object
370    Veeder
Name: New Last Name, dtype: object
John
Fonda
+------------------+
321    John D
Name: New First Name, dtype: object
321    Fonda
Name: New Last Name, dtype: object
John
Wart
+------------------+
432    John
Name: New First Name, dtype: object
432    Van Wart
Name: New Last Name, dtype: object
John L
Schermerhorn
+------------------+
310    John
Name: New First Name, dtype: object
310    Schermerhorn
Name: New Last Name, dtype: object
John
Fonda
+------------------+
321    John D
Name: New

Joseph
Owens
+------------------+
562    Joseph
Name: New First Name, dtype: object
562    Owen
Name: New Last Name, dtype: object
Philip
Jacob
+------------------+
615    Philip
Name: New First Name, dtype: object
615    Jacobs
Name: New Last Name, dtype: object
Joel
Harvey
+------------------+
601    Joel
Name: New First Name, dtype: object
601    Hervey
Name: New Last Name, dtype: object
Richard
Welch
+------------------+
622    Richard
Name: New First Name, dtype: object
622    Welsh
Name: New Last Name, dtype: object
Thomas
Janson
+------------------+
1053    Thomas
Name: New First Name, dtype: object
1053    Jansen
Name: New Last Name, dtype: object
John
Cluet
+------------------+
368    John
Name: New First Name, dtype: object
368    Clute
Name: New Last Name, dtype: object
John
Van allen
+------------------+
473    John
Name: New First Name, dtype: object
473    Van Allen
Name: New Last Name, dtype: object
John
Fonda
+------------------+
321    John D
Name: New First Name, dtyp

...

Joseph
Barden
+------------------+
1556    Joseph
Name: New First Name, dtype: object
1556    Braden
Name: New Last Name, dtype: object
John
McConnell
+------------------+
1574    John
Name: New First Name, dtype: object
1574    Connell
Name: New Last Name, dtype: object
Peter
Sisler
+------------------+
1800    Peter
Name: New First Name, dtype: object
1800    Seiler
Name: New Last Name, dtype: object
John
Kreamer
+------------------+
1595    John
Name: New First Name, dtype: object
1595    Creamer
Name: New Last Name, dtype: object
Michael
Defenderfer
+------------------+
1803    Michael
Name: New First Name, dtype: object
1803    Diffenderfer
Name: New Last Name, dtype: object
John
Sooder
+------------------+
1810    John
Name: New First Name, dtype: object
1810    Souder
Name: New Last Name, dtype: object
John
Leister
+------------------+
1815    John
Name: New First Name, dtype: object
1815    Lester
Name: New Last Name, dtype: object
Jacob
Weise
+------------------+
1819    Jacob

Adam
Whitman
+------------------+
1793    Adam
Name: New First Name, dtype: object
1793    Witman
Name: New Last Name, dtype: object
John Noble
Cummings
+------------------+
1869    John
Name: New First Name, dtype: object
1869    Cummings
Name: New Last Name, dtype: object
James
Thompson
+------------------+
1837    James
Name: New First Name, dtype: object
1837    Thomson
Name: New Last Name, dtype: object
Jonas
Phillips
+------------------+
1903    Jonas
Name: New First Name, dtype: object
1903    Philips
Name: New Last Name, dtype: object
Jonas
Phillips
+------------------+
1903    Jonas
Name: New First Name, dtype: object
1903    Philips
Name: New Last Name, dtype: object
Thomas
Wilson
+------------------+
1641    Thomas
Name: New First Name, dtype: object
1641    Willson
Name: New Last Name, dtype: object
Thomas
Wilson
+------------------+
1641    Thomas
Name: New First Name, dtype: object
1641    Willson
Name: New Last Name, dtype: object
Jonathan D
Sargeant
+------------------+

John
Robert
+------------------+
1515    John
Name: New First Name, dtype: object
1515    Roberts
Name: New Last Name, dtype: object
John
Rhoads
+------------------+
1364    John
Name: New First Name, dtype: object
1364    Rhoades
Name: New Last Name, dtype: object
Henry
Copper
+------------------+
1545    Henry
Name: New First Name, dtype: object
1545    Cooper
Name: New Last Name, dtype: object
John
Mcdonald
+------------------+
1546    John
Name: New First Name, dtype: object
1546    McDonald
Name: New Last Name, dtype: object
Philip
Marstetler
+------------------+
1548    Philip
Name: New First Name, dtype: object
1548    Marsteller
Name: New Last Name, dtype: object
Robert
McCluer
+------------------+
1560    Robert
Name: New First Name, dtype: object
1560    McClure
Name: New Last Name, dtype: object
James
Irwin
+------------------+
1088    James
Name: New First Name, dtype: object
1088    Erwin
Name: New Last Name, dtype: object
Peter
Whiteside
+------------------+
1567    Peter

William
Sharpe
+------------------+
1489    William
Name: New First Name, dtype: object
1489    Sharp
Name: New Last Name, dtype: object
Joseph
Ferre
+------------------+
1830    Joseph
Name: New First Name, dtype: object
1830    Ferree
Name: New Last Name, dtype: object
John
Coburne
+------------------+
1751    John
Name: New First Name, dtype: object
1751    Coburn
Name: New Last Name, dtype: object
William
Herper
+------------------+
1832    William
Name: New First Name, dtype: object
1832    Harper
Name: New Last Name, dtype: object
John Tin.
Brooke
+------------------+
1835    John
Name: New First Name, dtype: object
1835    Brooke
Name: New Last Name, dtype: object
Samuel
Futhey
+------------------+
1840    Samuel
Name: New First Name, dtype: object
1840    Futhy
Name: New Last Name, dtype: object
Jonas
Phillips
+------------------+
1903    Jonas
Name: New First Name, dtype: object
1903    Philips
Name: New Last Name, dtype: object
Philip
Dick
+------------------+
1799    Philip


John G
Frazier
+------------------+
792    John
Name: New First Name, dtype: object
792    Frazier
Name: New Last Name, dtype: object
John G
Frazier
+------------------+
792    John
Name: New First Name, dtype: object
792    Frazier
Name: New Last Name, dtype: object
John G
Frazier
+------------------+
792    John
Name: New First Name, dtype: object
792    Frazier
Name: New Last Name, dtype: object
Richard B
Hall
+------------------+
487    Richard
Name: New First Name, dtype: object
487    Hall
Name: New Last Name, dtype: object
Richard B
Hall
+------------------+
487    Richard
Name: New First Name, dtype: object
487    Hall
Name: New Last Name, dtype: object
Richard B
Hall
+------------------+
487    Richard
Name: New First Name, dtype: object
487    Hall
Name: New Last Name, dtype: object
Joseph
Dowson
+------------------+
788    Joseph
Name: New First Name, dtype: object
788    Dawson
Name: New Last Name, dtype: object
Richard
Parther
+------------------+
797    Richard
Name: New 

In [94]:
agg_debt.to_csv('data/final_agg_debt_cleaned.csv')